# Template for a Data Science & IA Bootcamp

---

## Sección 0.  Configuración inicial

In [ ]:
# Importación de librerías actualizadas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ydata_profiling import ProfileReport
import os

# Configuraciones generales
%matplotlib inline
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)

---

## Sección 1: Flexibilidad y Generalización

### 1.1. Parámetros configurables

Definimos los parámetros que el usuario puede configurar para la aplicación.

In [ ]:
# Parámetros configurables
RUTA_ARCHIVO = '<--->'     # Reemplaza con la ruta a tu archivo
SEPARADOR = ','                                     # Cambia el separador si es necesario (por ejemplo, ';' para archivos CSV europeos)
ENCODING = 'utf-8'                                  # Especifica la codificación del archivo si es necesario

### 1.2. Función para Cargar los datos
Creamos una función que detecte el formato del archivo y lo cargue adecuadamente.

In [ ]:
def cargar_datos(ruta_archivo, separador=',', encoding='utf-8'):
    """
    Carga datos desde un archivo CSV, Excel o JSON.
    
    Parámetros:
        ruta_archivo (str): Ruta al archivo de datos.
        separador (str): Separador utilizado en el archivo CSV.
        encoding (str): Codificación del archivo.
        
    Retorna:
        DataFrame de pandas con los datos cargados.
    """
    extension = os.path.splitext(ruta_archivo)[1].lower()
    if extension == '.csv':
        df = pd.read_csv(ruta_archivo, sep=separador, encoding=encoding)
    elif extension in ['.xls', '.xlsx']:
        df = pd.read_excel(ruta_archivo)
    elif extension == '.json':
        df = pd.read_json(ruta_archivo)
    else:
        raise ValueError("Formato de archivo no soportado. Utiliza CSV, Excel o JSON.")
    return df

### 1.3. Carga de Datos
Utilizamos la función para cargar datos.

In [ ]:
# Cargar datos
df = cargar_datos(RUTA_ARCHIVO, SEPARADOR, ENCODING)

---


## Sección 2: Análisis Exploratorio de Datos (EDA)

### 2.1. Información General del Dataset

Obtenemos información básica sobre el dataset.


In [ ]:
# Dimensiones del dataset
print(f"El dataset contiene {df.shape[0]} filas y {df.shape[1]} columnas.")

# Vista previa de los datos
df.head()


### 2.2. Tipos de Datos
Revisamos los tipos de datos de cada columna.

In [ ]:
# Tipos de datos
df.info()

### 2.3. Estadísticas Descriptivas


#### 2.3.1. Variables Numéticas

In [ ]:
# Estadísticas descriptivas de variables numéricas
df.describe()

#### 2.3.2. Variables Categóricas

In [ ]:
# Estadísticas descriptivas de variables categóricas
df.describe(include=['object', 'category'])


### 2.4 Valores Nulos
Creamos una función para resumir los valores nulos.

In [ ]:
def resumen_valores_nulos(df):
    nulos = df.isnull().sum()
    porcentaje = (nulos / len(df)) * 100
    resumen = pd.DataFrame({'Valores Nulos': nulos, 'Porcentaje (%)': porcentaje})
    resumen = resumen[resumen['Valores Nulos'] > 0].sort_values('Porcentaje (%)', ascending=False)
    return resumen

# Uso de la función
resumen_nulos = resumen_valores_nulos(df)
resumen_nulos


### 2.5. Visualizaciones Iniciales

#### 2.5.1. Variables Numéricas
Creamos una función para graficar histogramas y diagramas de caja para variables numéricas.

In [ ]:
def graficar_variables_numericas(df):
    numericas = df.select_dtypes(include=['int64', 'float64']).columns
    for col in numericas:
        fig, axes = plt.subplots(1, 2, figsize=(12, 4))
        sns.histplot(df[col].dropna(), ax=axes[0], kde=True, color='skyblue')
        axes[0].set_title(f'Histograma de {col}')
        sns.boxplot(x=df[col], ax=axes[1], color='lightgreen')
        axes[1].set_title(f'Diagrama de Caja de {col}')
        plt.tight_layout()
        plt.show()

# Uso de la función
graficar_variables_numericas(df)


#### 2.5.2. Variables Categóricas
Graficamos las frecuencias de las variables categóricas.

In [ ]:
def graficar_variables_categoricas(df):
    categoricas = df.select_dtypes(include=['object', 'category']).columns
    for col in categoricas:
        plt.figure(figsize=(8, 4))
        orden = df[col].value_counts().index
        sns.countplot(data=df, x=col, order=orden, palette='viridis')
        plt.xticks(rotation=45)
        plt.title(f'Conteo de {col}')
        plt.tight_layout()
        plt.show()

# Uso de la función
graficar_variables_categoricas(df)


### 2.6. Matriz de Correlación
Calculamos y visualizamos la matriz de correlación.

In [ ]:
# Matriz de correlación
corr_matrix = df.select_dtypes(include=['int64', 'float64']).corr()

# Visualización
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Matriz de Correlación')
plt.show()


### 2.7. Detección de Valores Atípicos
Implementamos la detección de outliers usando el rango intercuartílico (IQR).

In [ ]:
def detectar_outliers_iqr(df):
    numericas = df.select_dtypes(include=['int64', 'float64']).columns
    outliers = {}
    for col in numericas:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        limite_inferior = Q1 - 1.5 * IQR
        limite_superior = Q3 + 1.5 * IQR
        outliers_col = df[(df[col] < limite_inferior) | (df[col] > limite_superior)]
        outliers[col] = outliers_col
        print(f"{col}: {len(outliers_col)} outliers detectados.")
    return outliers

# Uso de la función
outliers_detectados = detectar_outliers_iqr(df)


---


## Sección 3: Preprocesamiento de Datos

#### 3.1. Manejo de Valores Nulos
Implemantamos estrategias para manejar valores nulos en nuestros datos.

##### 3.1.1. Imputación de Variables Numéricas

In [ ]:
def imputar_numericas(df, metodo='media'):
    numericas = df.select_dtypes(include=['int64', 'float64']).columns
    for col in numericas:
        if df[col].isnull().sum() > 0:
            if metodo == 'media':
                valor_imputacion = df[col].mean()
            elif metodo == 'mediana':
                valor_imputacion = df[col].median()
            df[col].fillna(valor_imputacion, inplace=True)
    return df

# Uso de la función
df = imputar_numericas(df, metodo='media')


##### 3.1.2. Imputación de Variables Categóricas

In [ ]:
def imputar_categoricas(df, metodo='moda'):
    categoricas = df.select_dtypes(include=['object', 'category']).columns
    for col in categoricas:
        if df[col].isnull().sum() > 0:
            if metodo == 'moda':
                valor_imputacion = df[col].mode()[0]
            elif metodo == 'desconocido':
                valor_imputacion = 'Desconocido'
            df[col].fillna(valor_imputacion, inplace=True)
    return df

# Uso de la función
df = imputar_categoricas(df, metodo='moda')


### 3.2. Codificación de Variables Categóricas
Implementamos la codificación de variables categóricas utilizando One-Hot Encoding.

In [ ]:
def codificar_categoricas(df):
    categoricas = df.select_dtypes(include=['object', 'category']).columns
    df = pd.get_dummies(df, columns=categoricas, drop_first=True)
    return df

# Uso de la función
df = codificar_categoricas(df)

### 3.3. Escalado de Variables Numéricas
Aplicamos escalado estándar a las variables numéticas.

In [ ]:
from sklearn.preprocessing import StandardScaler

def escalar_numericas(df):
    numericas = df.select_dtypes(include=['int64', 'float64']).columns
    scaler = StandardScaler()
    df[numericas] = scaler.fit_transform(df[numericas])
    return df

# Uso de la función
df = escalar_numericas(df)


---

## Sección 4: Automatización y Personalización

### 4.1 Parámetros adicionales configurables

In [ ]:
# Umbral para eliminar columnas con alto porcentaje de valores nulos
UMBRALES_NULOS = 50  # Porcentaje máximo permitido

# Decisión de eliminar o imputar valores nulos
ELIMINAR_COLUMNAS_NULOS = False  # True para eliminar, False para imputar


### 4.2. Implementación de Lógica Condicional
Podemos incluir lógica para eliminar columnas con muchos valores nulos.

In [ ]:
def manejar_columnas_nulas(df, umbral=50, eliminar=False):
    resumen_nulos = resumen_valores_nulos(df)
    columnas_a_eliminar = resumen_nulos[resumen_nulos['Porcentaje (%)'] > umbral].index
    if eliminar:
        df.drop(columns=columnas_a_eliminar, inplace=True)
        print(f"Se eliminaron {len(columnas_a_eliminar)} columnas con más del {umbral}% de valores nulos.")
    else:
        print(f"Columnas con más del {umbral}% de valores nulos: {list(columnas_a_eliminar)}")
    return df

# Uso de la función
df = manejar_columnas_nulas(df, umbral=UMBRALES_NULOS, eliminar=ELIMINAR_COLUMNAS_NULOS)


## Sección 5: Documentación y Reportes

**Instalación de pandas-profiling**

Si aún no lo tienes instalado, puedes instalarlo usando:

```bash
pip install pandas-profiling
```

### Generación del Reporte

In [ ]:
# Generación del reporte
perfil = ProfileReport(df, title='Reporte del Dataset', explorative=True)

# Mostrar el reporte en el notebook
perfil.to_notebook_iframe()

### Exportar el Reporte a HTML

In [ ]:
# Exportar el reporte a un archivo HTML
perfil.to_file('reporte_dataset.html')